In [15]:
import requests
from od3d.datasets.objectron.enum import OBJECTRON_CATEGORIES

from pathlib import Path

path_objectron_raw = Path("OBJECTRON")

for category in list(OBJECTRON_CATEGORIES):
    category_str = f"{category}"
    public_url = "https://storage.googleapis.com/objectron"
    blob_path = public_url + f"/v1/index/{category_str}_annotations"
    video_ids = requests.get(blob_path).text
    video_ids = video_ids.split('\n')

video_ids_lengths = [len(video_id) for video_id in video_ids]

In [23]:
annotation_fpath = 'OBJECTRON/bike/batch_0_0/annotation.pbdata'

from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol

with open(annotation_fpath, "rb") as pb:
    sequence = annotation_protocol.Sequence()
    sequence.ParseFromString(pb.read())
    
    rotation =  sequence.objects[0].rotation
    translation =  sequence.objects[0].translation
    scale =  sequence.objects[0].scale
    type =  sequence.objects[0].type
    
    tform_obj = 
    category =  sequence.objects[0].category

    # sequence.objects[object_id].rotation
    # sequence.objects[object_id].translation
    # sequence.objects[object_id].scale
    # sequence.objects[object_id].category
    # sequence.objects[object_id].type

    # sequence.frame_annotations[frame_id].camera.transform
    # sequence.frame_annotations[frame_id].camera.projection_matrix
    # sequence.frame_annotations[frame_id].camera.view_matrix
    # sequence.frame_annotations[frame_id].camera.intrinsics        
    
    # sequence.frame_annotations[frame_id].raw_feature_points.point[point_id].x/.y/.z
    # sequence.frame_annotations[frame_id].annotations[object_id].keypoints[keypoint_id].point_2d.x/.y/.depth
    # sequence.frame_annotations[frame_id].annotations[object_id].keypoints[keypoint_id].point_3d.x/.y/.z
    
rotation


[0.5427104830741882, -1.0278942141894731e-08, 0.8979186415672302, -2.630373074907766e-08, 1.0, -0.01380867138504982, -0.8399199843406677, 3.484810662257587e-08, 0.43994489312171936]

In [2]:
annotation_fpath = 'OBJECTRON/bike/batch_0_0/annotation.pbdata'

from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol

with open(annotation_fpath, "rb") as pb:
    sequence = annotation_protocol.Sequence()
    sequence.ParseFromString(pb.read())
    
len( sequence.frame_annotations)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


318

In [31]:
meta_fpath = 'OBJECTRON/bike/batch_0_0/meta.pbdata'

from od3d.datasets.objectron.schema import a_r_capture_metadata_pb2 as ar_metadata_protocol
import struct
import numpy as np
sequence_geometry = []
with open(meta_fpath, 'rb') as pb:
    proto_buf = pb.read()

    i = 0
    frame_number = 0
    
    points = []
    while i < len(proto_buf)//10:
        # print(i)
        #print(len(proto_buf))
        # Read the first four Bytes in little endian '<' integers 'I' format
        # indicating the length of the current message.
        msg_len = struct.unpack('<I', proto_buf[i:i + 4])[0]
        i += 4
        message_buf = proto_buf[i:i + msg_len]
        i += msg_len
        # ARMeshGeometry
        frame_data = ar_metadata_protocol.ARFrame()
        frame_data.ParseFromString(message_buf)
        # print(frame_data.raw_feature_points.point)
        current_points = [np.array([v.x, v.y, v.z])
                          for v in frame_data.raw_feature_points.point]
        current_points = np.array(current_points)

        points.append(current_points)

points = np.concatenate(points, axis=0)

In [33]:
import open3d as o3d
import numpy as np

# Create an Open3D point cloud object
pcd = o3d.geometry.PointCloud()

# Convert your point cloud numpy array to Open3D format
pcd.points = o3d.utility.Vector3dVector(points)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd])

In [28]:
annotation_fpath = 'OBJECTRON/bike/batch_0_0/annotation.pbdata'

from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol

with open(annotation_fpath, "rb") as pb:
    sequence = annotation_protocol.Sequence()

1

In [17]:
import cv2
from pathlib import Path 

# Path to the .MOV video file
seq_name = "batch_0_0"
category = "bike"
video_path = f"OBJECTRON/{category}/{seq_name}/video.MOV"
frames_path = Path(f"OBJECTRON/{category}/{seq_name}/frames")
# Create a VideoCapture object
cap = cv2.VideoCapture(video_path)

# Check if the video was opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

frame_id = 0
# Loop through the video frames
while True:
    # Read a frame
    ret, frame = cap.read()
    
    # If we got a valid frame, process it
    if ret:
        fpath = frames_path.joinpath(f"{frame_id}.jpg")
        fpath.parent.mkdir(exist_ok=True, parents=True)
        # Display the frame (optional)
        # cv2.imshow('Frame', frame)
        cv2.imwrite(filename=fpath, img=frame)
        
        frame_id += 1
        # Wait for 25ms and check if the user pressed the 'q' key to exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        # If no more frames, break the loop
        break

# Release the VideoCapture object and close any open windows
cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt



In [11]:
frame

In [13]:
import requests
from od3d.datasets.objectron.enum import OBJECTRON_CATEGORIES

from pathlib import Path

path_objectron_raw = Path("OBJECTRON")

for category in list(OBJECTRON_CATEGORIES):
    category_str = f"{category}"
    public_url = "https://storage.googleapis.com/objectron"
    blob_path = public_url + f"/v1/index/{category_str}_annotations"
    video_ids = requests.get(blob_path).text
    video_ids = video_ids.split('\n')
    
    # Download the first ten videos in cup test dataset
    for i in range(1):
        video_id = video_ids[i] # "bike/batch-4/38" "bike/batch-4/43" # video_ids[i]
        if len(video_id) == 0:
            continue
        video_filename = public_url + f"/videos/" + video_id + "/video.MOV"
        metadata_filename = public_url + "/videos/" + video_id + "/geometry.pbdata"
        annotation_filename = public_url + "/annotations/" + video_id + ".pbdata"
        
        seq_name = '_'.join(video_id.split('/')[1:]).replace('-', '_')
        
        annotation = requests.get(annotation_filename)
        path_objectron_raw.joinpath(category_str, seq_name).mkdir(exist_ok=True, parents=True)
        annotation_fpath = path_objectron_raw.joinpath(category_str, seq_name, "annotation.pbdata")
        file = open(annotation_fpath, "wb")
        file.write(annotation.content)
        file.close()
        
        metadata = requests.get(metadata_filename)
        metadata_fpath = path_objectron_raw.joinpath(category_str, seq_name, "meta.pbdata")
        file = open(metadata_fpath, "wb")
        file.write(metadata.content)
        file.close()
    
        # video.content contains the video file.
        video = requests.get(video_filename)
        video_fpath = path_objectron_raw.joinpath(category_str, seq_name, "video.MOV")
        file = open(video_fpath, "wb")
        file.write(video.content)
        file.close()


In [31]:
public_url = "https://storage.googleapis.com/objectron"
blob_path = public_url + "/v1/index/cup_annotations_test"
video_ids = requests.get(blob_path).text
# bike/batch-4/38
# bike/batch-4/43
video_ids = video_ids.split('\n')
# Download the first ten videos in cup test dataset
for i in range(1):
    video_id = "bike/batch-4/38" # "bike/batch-4/38" "bike/batch-4/43" # video_ids[i]
    video_filename = public_url + "/videos/" + video_id + "/video.MOV"
    metadata_filename = public_url + "/videos/" + video_id + "/geometry.pbdata"
    annotation_filename = public_url + "/annotations/" + video_id + ".pbdata"
    # video.content contains the video file.
    video = requests.get(video_filename)
    metadata = requests.get(metadata_filename)
    annotation = requests.get(annotation_filename)
    file = open(f"{video_id.replace('/', '')}_example.MOV", "wb")
    file.write(video.content)
    file.close()

ModuleNotFoundError: No module named 'objectron'

In [11]:
#from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol
import requests
annotation = requests.get('https://storage.googleapis.com/objectron/annotations/cup/batch-16/47.pbdata')
#sequence = annotation_protocol.Sequence()
#sequence.ParseFromString(annotation.read())
#with open('https://storage.googleapis.com/objectron/annotations/cup/batch-16/47.pbdata', 'rb') as pb:
#    pb.write('x.pb')
    #sequence = annotation_protocol.Sequence()
    #sequence.ParseFromString(pb.read())

In [20]:
from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol

sequence = annotation_protocol.Sequence()
sequence.ParseFromString(annotation.content)

173053

In [25]:
sequence.objects
sequence.frame_annotations[0]


annotations {
  keypoints {
    point_3d {
      x: -0.025665238
      y: -0.002877787
      z: -0.23401287
    }
    point_2d {
      x: 0.48864055
      y: 0.39443862
      depth: 0.23401289
    }
  }
  keypoints {
    id: 1
    point_3d {
      x: -0.009848198
      y: -0.03808612
      z: -0.2971413
    }
    point_2d {
      x: 0.35811102
      y: 0.45919612
      depth: 0.29714128
    }
  }
  keypoints {
    id: 2
    point_3d {
      x: 0.03839782
      y: -0.037761655
      z: -0.24642426
    }
    point_2d {
      x: 0.3297581
      y: 0.6189336
      depth: 0.24642424
    }
  }
  keypoints {
    id: 3
    point_3d {
      x: -0.08954753
      y: -0.03799315
      z: -0.22132564
    }
    point_2d {
      x: 0.30890083
      y: 0.14494479
      depth: 0.22132562
    }
  }
  keypoints {
    id: 4
    point_3d {
      x: -0.04130151
      y: -0.037668686
      z: -0.17060861
    }
    point_2d {
      x: 0.25315908
      y: 0.2821169
      depth: 0.1706086
    }
  }
  keypoints 

In [26]:
sequence

objects {
  category: "chair"
  type: BOUNDING_BOX
  rotation: 1.0
  rotation: 0.0
  rotation: 0.0
  rotation: 0.0
  rotation: 1.0
  rotation: 0.0
  rotation: 0.0
  rotation: 0.0
  rotation: 1.0
  translation: -0.01807152
  translation: -0.15593788
  translation: -0.20586886
  scale: 0.07
  scale: 0.11
  scale: 0.07
  keypoints {
  }
  keypoints {
    x: -0.5
    y: -0.5
    z: -0.5
  }
  keypoints {
    x: -0.5
    y: -0.5
    z: 0.5
  }
  keypoints {
    x: -0.5
    y: 0.5
    z: -0.5
  }
  keypoints {
    x: -0.5
    y: 0.5
    z: 0.5
  }
  keypoints {
    x: 0.5
    y: -0.5
    z: -0.5
  }
  keypoints {
    x: 0.5
    y: -0.5
    z: 0.5
  }
  keypoints {
    x: 0.5
    y: 0.5
    z: -0.5
  }
  keypoints {
    x: 0.5
    y: 0.5
    z: 0.5
  }
}
frame_annotations {
  annotations {
    keypoints {
      point_3d {
        x: -0.025665238
        y: -0.002877787
        z: -0.23401287
      }
      point_2d {
        x: 0.48864055
        y: 0.39443862
        depth: 0.23401289
      }

In [13]:
from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol

with open('https://storage.googleapis.com/objectron/annotations/cup/batch-16/47.pbdata', 'rb') as pb:
    #pb.write('x.pb')
    sequence = annotation_protocol.Sequence()
    sequence.ParseFromString(pb.read())
    
#sequence = annotation_protocol.Sequence()
#sequence.ParseFromString(annotation.read())

FileNotFoundError: [Errno 2] No such file or directory: 'https://storage.googleapis.com/objectron/annotations/cup/batch-16/47.pbdata'

In [6]:
from od3d.datasets.objectron.schema import annotation_data_pb2 as annotation_protocol


ModuleNotFoundError: No module named 'a_r_capture_metadata_pb2'